In [ ]:
!pip install transformers torch mysql-connector-python streamlit flask pyngrok

ERROR: Could not find a version that satisfies the requirement localtunnel (from versions: none)
ERROR: No matching distribution found for localtunnel


In [ ]:
# Grant access to hugging face
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import mysql.connector
import torch


In [ ]:
model_name = 'google/flan-t5-base' # st3ml llama2-8b-instruct 7sn
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
''' Connect to MySQL DB '''

# def fetch_reservations(user, password, host, db_name):
#   db_config = {
#       'user': user,
#       'password': password,
#       'host': host,
#       'database': db_name,
#       'raise_on_warnings': True
#   }

#   conn = mysql.connector.connect(**db_config)
#   cursor = conn.cursor()


#   cursor.execute("select * from évènements")
#   reservations = cursor.fetchall()

#   conn.close()

#   documents = []
#   for reservation in reservations:
#     id, date, description, _, lieu, nom, utilisateur_id = reservation
#     doc = f"Reservation ID is : {id}, in the date {date}, the reservation description is {description}, and this happen in {lieu}, Created by {utilisateur_id}"
#     documents.append(doc)

#     return documents

def fetch_reservations():
    records = [
        (1, '2025-01-15', 'Annual Tech Conference focused on AI and Cybersecurity', 'https://example.com/tech2025.jpg', 'San Francisco', 'Tech Innovators 2025', 201),
        (2, '2025-02-10', 'A music festival featuring top artists from around the globe', 'https://example.com/musicfest.jpg', 'London', 'Global Music Fest', 202),
        (3, '2025-03-22', 'An exhibition showcasing modern art from emerging artists', 'https://example.com/artexpo.jpg', 'Paris', 'Modern Art Expo', 203),
        (4, '2025-04-05', 'Marathon for raising awareness on climate change', 'https://example.com/marathon.jpg', 'New York', 'Green Earth Marathon', 204),
        (5, '2025-05-18', 'International book fair with author signings and workshops', 'https://example.com/bookfair.jpg', 'Berlin', 'World Book Fair 2025', 205),
        (6, '2025-06-25', 'Conference on innovations in fintech and blockchain technology', 'https://example.com/fintech.jpg', 'Singapore', 'Fintech Future Summit', 206),
        (7, '2025-07-12', 'Gastronomy festival celebrating world cuisines and culture', 'https://example.com/foodfest.jpg', 'Tokyo', 'Global Food Festival', 207),
        (8, '2025-08-08', 'Concert featuring legendary rock bands from the 80s', 'https://example.com/rockconcert.jpg', 'Los Angeles', 'Rock Revival 2025', 208),
        (9, '2025-09-30', 'Startup competition with prizes for innovative projects', 'https://example.com/startup.jpg', 'Dubai', 'Startup Challenge 2025', 209),
        (10, '2025-10-20', 'Workshop on the latest trends in web development', 'https://example.com/webworkshop.jpg', 'Sydney', 'Web Dev Workshop 2025', 210)
    ]

    # Convert records into a list of strings (documents)
    documents = []
    for reservation in records:
        id, date, description, _, lieu, nom, utilisateur_id = reservation
        doc = f"Reservation ID: {id}, Date: {date}, Description: {description}, Location: {lieu}, Event Name: {nom}, Organizer ID: {utilisateur_id}"
        documents.append(doc)

    return documents


In [ ]:
def retrieve_documents(query, documents, top_k=3):
    """
    Retrieves the top-k most relevant documents based on cosine similarity.
    """
    # Use TF-IDF to vectorize the documents and query
    vectorizer = TfidfVectorizer()  # Initialize the TF-IDF vectorizer
    doc_vectors = vectorizer.fit_transform(documents)  # Convert documents into TF-IDF vectors
    query_vector = vectorizer.transform([query])  # Convert the query into a TF-IDF vector

    # Compute cosine similarity between the query and documents
    similarities = cosine_similarity(query_vector, doc_vectors).flatten()  # Get similarity scores

    # Get the indices of the top-k most similar documents
    top_k_indices = np.argsort(similarities)[-top_k:][::-1]  # Sort indices by similarity (descending)

    # Return the top-k documents
    top_k_documents = [documents[i] for i in top_k_indices]  # Fetch the top-k documents
    return top_k_documents

**Generate Response**

In [ ]:
def generate_response(query, retrieved_documents):
    """
    Generates a response using the retrieved documents and query.
    """
    # Combine the retrieved documents into a single context
    context = " ".join(retrieved_documents)  # Join the documents into a single string

    # Combine the query and context
    input_text = f"Question: {query}\nContext: {context}"  # Format the input for the model

    # Tokenize the input
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate the response
    outputs = model.generate(inputs["input_ids"], max_length=100)  # Generate text using FLAN-T5
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)  # Decode the generated text

    return response  # Return the generated response

reservations = fetch_reservations()


**Chat bot inference**




In [ ]:
def rag_chatbot():
    """
    Runs the RAG chatbot in a loop, allowing the user to ask questions about reservations.
    """
    print("Welcome to the RAG Reservation Chatbot! How can I assist you today?")

    while True:
        # Get user input
        user_input = input("You: ")

        # Exit the chat if the user says "exit" or "quit"
        if user_input.lower() in ["exit", "quit"]:
            print("Chatbot: Goodbye! Have a great day.")
            break

        # Retrieve relevant documents
        retrieved_docs = retrieve_documents(user_input, reservations, top_k=3)  # Fetch top-3 relevant docs

        # Generate a response using the retrieved documents
        response = generate_response(user_input, retrieved_docs)  # Generate a response using FLAN-T5
        print(f"Chatbot: {response}")

# Run the chatbot
rag_chatbot()

**Flask API**

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import threading

app = Flask(__name__)
# Set your ngrok authtoken
ngrok.set_auth_token("NG_TOKEN")
print("Ngrok authtoken set successfully.")

@app.route('/chat', methods=['POST'])
def chat():
    """
    Flask API endpoint for handling chat requests.
    """
    # Get the user query from the request body
    data = request.json  # Parse the JSON request body
    query = data.get('query')  # Extract the 'query' field

    if not query:
        return jsonify({"error": "Query is required"}), 400  # Return an error if no query is provided

    # Retrieve relevant documents
    retrieved_docs = retrieve_documents(query, reservations, top_k=3)  # Fetch top-3 relevant docs

    # Generate a response using the retrieved documents
    response = generate_response(query, retrieved_docs)  # Generate a response using FLAN-T5

    # Return the response as JSON
    return jsonify({"response": response})

# Function to run the Flask app
def run_flask():
    # Start ngrok tunnel for Flask
    flask_ngrok_tunnel = ngrok.connect(5000)
    flask_public_url = flask_ngrok_tunnel.public_url
    print(f" * Flask API is running at: {flask_public_url}")

    # Write the Flask ngrok URL to a file
    with open("flask_ngrok_url.txt", "w") as f:
        f.write(flask_public_url)

    # Run the Flask app (blocking call)
    app.run(host='0.0.0.0', port=5000, debug=True)  # Enable debug mode for detailed error logs

# Run Flask app in a separate thread
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

# Wait for Flask app to start
import time
time.sleep(5)


**Chat Interface**

In [ ]:
%%writefile app.py
import streamlit as st
import requests

# Read the Flask ngrok URL from the file
with open("flask_ngrok_url.txt", "r") as f:
    flask_ngrok_url = f.read().strip()

# Define the Flask API endpoint using the ngrok URL
API_URL = f"{flask_ngrok_url}/chat"

# Streamlit app title
st.title("BS-Enventos Reservation Chatbot")

# Initialize session state to store chat history
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

# Function to send a query to the Flask API
def send_query(query):
    try:
        response = requests.post(API_URL, json={"query": query})
        if response.status_code == 200:
            return response.json().get("response", "No response from the chatbot.")
        else:
            return f"Error: {response.status_code} - {response.text}"
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Display chat history
for message in st.session_state.chat_history:
    with st.chat_message(message["role"]):
        st.write(message["content"])

# Input field for user query
user_query = st.chat_input("Type your message here...")

# Handle user input
if user_query:
    st.session_state.chat_history.append({"role": "user", "content": user_query})
    with st.chat_message("user"):
        st.write(user_query)
    chatbot_response = send_query(user_query)
    st.session_state.chat_history.append({"role": "assistant", "content": chatbot_response})
    with st.chat_message("assistant"):
        st.write(chatbot_response)

Overwriting app.py


In [ ]:
# Run Streamlit app in the background
import subprocess
streamlit_process = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8502"])

# Wait for Streamlit app to start
import time
time.sleep(5)

print("Streamlit app is running in the background.")

Streamlit app is running in the background.


In [ ]:
ngrok.kill()
print("All existing ngrok tunnels have been killed.")

All existing ngrok tunnels have been killed.


In [ ]:
from pyngrok import ngrok

# Expose Streamlit app with ngrok
streamlit_ngrok_tunnel = ngrok.connect(8502)
streamlit_public_url = streamlit_ngrok_tunnel.public_url
print(f" * Streamlit app is running at: {streamlit_public_url}")

 * Streamlit app is running at: https://4ef7-34-87-96-230.ngrok-free.app
